In [5]:
import google.protobuf

print(google.protobuf.__version__)


4.23.2


In [6]:
pip show protobuf


Name: protobuf
Version: 4.23.2
Summary: 
Home-page: https://developers.google.com/protocol-buffers/
Author: protobuf@googlegroups.com
Author-email: protobuf@googlegroups.com
License: 3-Clause BSD License
Location: c:\users\asus\appdata\local\programs\python\python38\lib\site-packages
Requires: 
Required-by: googleapis-common-protos, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-intel, tensorflow-metadata
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip uninstall protobuf


^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install protobuf==3.20.0


In [4]:
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [14]:
# Load the recipe data
recipes_df = pd.read_csv('recipes.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'recipes.csv'

In [ ]:
# Preprocess the recipe data
recipes_df['ingredients'] = recipes_df['ingredients'].str.lower()
recipes_df['instructions'] = recipes_df['instructions'].str.lower()

In [ ]:
# Create the user and item feature columns
user_id = tf.feature_column.categorical_column_with_vocabulary_list('user_id', recipes_df['user_id'].unique())
recipe_id = tf.feature_column.categorical_column_with_vocabulary_list('recipe_id', recipes_df['recipe_id'].unique())

user_embedding = tf.feature_column.embedding_column(user_id, dimension=32)
recipe_embedding = tf.feature_column.embedding_column(recipe_id, dimension=32)

In [ ]:
# Create the model
model = tf.keras.Sequential([
    user_embedding,
    recipe_embedding,
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1),
              loss=tf.keras.losses.MeanSquaredError())

In [ ]:
# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((dict(recipes_df), recipes_df['rating']))
train_dataset = dataset.shuffle(len(recipes_df)).batch(32)

In [ ]:
# Train the model
model.fit(train_dataset, epochs=10)

In [ ]:
# Create the retrieval model
retrieval_model = tf.keras.Sequential([user_embedding, tf.keras.layers.Dense(16)])
retrieval_model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [ ]:
# Set up the retrieval task
index = tfrs.layers.factorized_top_k.BruteForce(retrieval_model)
index.index_from_dataset(tf.data.Dataset.from_tensor_slices((dict(recipes_df), recipes_df['recipe_id'])))

In [ ]:
# Retrieve the top 5 recommendations for a given user
user_id = "user1"
user_row = recipes_df[recipes_df['user_id'] == user_id]
user_data = tf.data.Dataset.from_tensor_slices(dict(user_row))
top_k = index(tf.constant([user_id]), k=5)

In [ ]:
# Print the top recommendations
print(f"Top 5 recommendations for user {user_id}:")
for i, recipe_id in enumerate(top_k[1][0]):
    recipe_title = recipes_df[recipes_df['recipe_id'] == recipe_id]['title'].values[0]
    print(f"{i + 1}. {recipe_title}")